### Import Required Libraries and Set Up Environment Variables

In [937]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [938]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}CME?StartDate={startDate}&EndDate={endDate}&api_key={NASA_API_KEY}"

In [939]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)
cme_response

<Response [200]>

In [940]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [941]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json, indent=4))

[
    {
        "activityID": "2024-11-02T06:24:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2024-11-02T06:24Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "Faint, narrow CME visible to the SW in SOHO LASCO C2/C3 and STEREO A COR2. There are potential source candidate eruptions seen on the disk in available EUV imagery, including a faint dimming seen in GOES SUVI 195/284 and STEREO A EUV 195 around S40W10 starting at 2024-11-02T05:00Z. Dimming and moving field lines can also be seen south of AR 3869 (approx. S20W50) starting at 2024-11-02T02:52Z. The eruption is not seen in SDO AIA imagery due to a data gap from approximately 2024-11-01T22:00Z to 2024-11-02T0

In [942]:
# Convert cme_json to a Pandas DataFrame 
df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
df=df[['activityID', 'startTime', 'linkedEvents']]
df.head()

,activityID,startTime,linkedEvents
0,2024-11-02T06:24:00-CME-001,2024-11-02T06:24Z,None
1,2024-11-02T09:12:00-CME-001,2024-11-02T09:12Z,None
2,2024-11-03T13:25:00-CME-001,2024-11-03T13:25Z,None
3,2024-11-03T15:27:00-CME-001,2024-11-03T15:27Z,[{'activityID': '2024-11-03T15:13:00-FLR-001'}]
4,2024-11-04T00:24:00-CME-001,2024-11-04T00:24Z,None


In [943]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
df_cleaned = df.dropna()
df_cleaned.head()

,activityID,startTime,linkedEvents
3,2024-11-03T15:27:00-CME-001,2024-11-03T15:27Z,[{'activityID': '2024-11-03T15:13:00-FLR-001'}]
5,2024-11-04T01:36:00-CME-001,2024-11-04T01:36Z,[{'activityID': '2024-11-04T01:05:00-FLR-001'}...
6,2024-11-04T13:23:00-CME-001,2024-11-04T13:23Z,[{'activityID': '2024-11-04T11:46:00-FLR-001'}]
7,2024-11-04T17:24:00-CME-001,2024-11-04T17:24Z,[{'activityID': '2024-11-04T15:26:00-FLR-001'}...
8,2024-11-05T00:09:00-CME-001,2024-11-05T00:09Z,[{'activityID': '2024-11-07T14:29:00-IPS-001'}]


In [944]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 
# Initialize an empty list to store the expanded rows
# Iterate over each index in the DataFrame
# Create a new DataFrame from the expanded rows

expanded_rows = []
for index, row in df_cleaned.iterrows():
    activityID = row['activityID']
    startTime = row['startTime']
    linkedEvents = row['linkedEvents']
    for event in linkedEvents:
        expanded_rows.append({'activityID': activityID, 'startTime': startTime, 'linkedEvents': linkedEvents})
    
df_expanded = pd.DataFrame(expanded_rows)
print(df_expanded.head())


                    activityID          startTime  \
0  2024-11-03T15:27:00-CME-001  2024-11-03T15:27Z   
1  2024-11-04T01:36:00-CME-001  2024-11-04T01:36Z   
2  2024-11-04T01:36:00-CME-001  2024-11-04T01:36Z   
3  2024-11-04T13:23:00-CME-001  2024-11-04T13:23Z   
4  2024-11-04T17:24:00-CME-001  2024-11-04T17:24Z   

                                        linkedEvents  
0    [{'activityID': '2024-11-03T15:13:00-FLR-001'}]  
1  [{'activityID': '2024-11-04T01:05:00-FLR-001'}...  
2  [{'activityID': '2024-11-04T01:05:00-FLR-001'}...  
3    [{'activityID': '2024-11-04T11:46:00-FLR-001'}]  
4  [{'activityID': '2024-11-04T15:26:00-FLR-001'}...  


In [945]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
        try:
                activityID = input_dict.get('activityID', None)
                return activityID
        except (ValueError, TypeError) as e:
        # Log the error or print it for debugging
                print(f"Error processing input dictionary: {input_dict}. Error: {e}")
        return None

example_row = df.loc[0, 'linkedEvents']
if example_row:
        activity_id =extract_activityID_from_dict(example_row[0])
        print(activity_id)
else:
        print("no linked events found in the example row.")

# Log the error or print it for debugging
print(activityID)



no linked events found in the example row.
2024-11-27T02:48:00-CME-001


In [946]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
#print(df['linkedEvents'].head())
df.loc[:, 'GST_ActivityID'] = df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x[0]) if isinstance (x, list) and len(x) > 0 else None)
print(df.head(10))

                    activityID          startTime  \
0  2024-11-02T06:24:00-CME-001  2024-11-02T06:24Z   
1  2024-11-02T09:12:00-CME-001  2024-11-02T09:12Z   
2  2024-11-03T13:25:00-CME-001  2024-11-03T13:25Z   
3  2024-11-03T15:27:00-CME-001  2024-11-03T15:27Z   
4  2024-11-04T00:24:00-CME-001  2024-11-04T00:24Z   
5  2024-11-04T01:36:00-CME-001  2024-11-04T01:36Z   
6  2024-11-04T13:23:00-CME-001  2024-11-04T13:23Z   
7  2024-11-04T17:24:00-CME-001  2024-11-04T17:24Z   
8  2024-11-05T00:09:00-CME-001  2024-11-05T00:09Z   
9  2024-11-05T07:36:00-CME-001  2024-11-05T07:36Z   

                                        linkedEvents  \
0                                               None   
1                                               None   
2                                               None   
3    [{'activityID': '2024-11-03T15:13:00-FLR-001'}]   
4                                               None   
5  [{'activityID': '2024-11-04T01:05:00-FLR-001'}...   
6    [{'activityID': '20

In [947]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
df = df.dropna(subset=["GST_ActivityID"])
df.head()



,activityID,startTime,linkedEvents,GST_ActivityID
3,2024-11-03T15:27:00-CME-001,2024-11-03T15:27Z,[{'activityID': '2024-11-03T15:13:00-FLR-001'}],2024-11-03T15:13:00-FLR-001
5,2024-11-04T01:36:00-CME-001,2024-11-04T01:36Z,[{'activityID': '2024-11-04T01:05:00-FLR-001'}...,2024-11-04T01:05:00-FLR-001
6,2024-11-04T13:23:00-CME-001,2024-11-04T13:23Z,[{'activityID': '2024-11-04T11:46:00-FLR-001'}],2024-11-04T11:46:00-FLR-001
7,2024-11-04T17:24:00-CME-001,2024-11-04T17:24Z,[{'activityID': '2024-11-04T15:26:00-FLR-001'}...,2024-11-04T15:26:00-FLR-001
8,2024-11-05T00:09:00-CME-001,2024-11-05T00:09Z,[{'activityID': '2024-11-07T14:29:00-IPS-001'}],2024-11-07T14:29:00-IPS-001


In [948]:
# print out the datatype of each column in this DataFrame:
print(df.dtypes)

activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object


In [949]:
print(df.columns)

Index(['activityID', 'startTime', 'linkedEvents', 'GST_ActivityID'], dtype='object')


In [950]:
# Convert the 'GST_ActivityID' column to string format 
df['GST_ActivityID'] = df['GST_ActivityID'].astype(str)
# Convert startTime to datetime format  
df['startTime'] = pd.to_datetime(df['startTime'])
# Rename startTime to startTime_CME and activityID to cmeID
df = df.rename(columns={'startTime':'startTime+cme','activityID':'cmeID'})
# Drop linkedEvents
df = df.drop(columns=['linkedEvents'])
# Verify that all steps were executed correctly
print(df.head())


                         cmeID             startTime+cme  \
3  2024-11-03T15:27:00-CME-001 2024-11-03 15:27:00+00:00   
5  2024-11-04T01:36:00-CME-001 2024-11-04 01:36:00+00:00   
6  2024-11-04T13:23:00-CME-001 2024-11-04 13:23:00+00:00   
7  2024-11-04T17:24:00-CME-001 2024-11-04 17:24:00+00:00   
8  2024-11-05T00:09:00-CME-001 2024-11-05 00:09:00+00:00   

                GST_ActivityID  
3  2024-11-03T15:13:00-FLR-001  
5  2024-11-04T01:05:00-FLR-001  
6  2024-11-04T11:46:00-FLR-001  
7  2024-11-04T15:26:00-FLR-001  
8  2024-11-07T14:29:00-IPS-001  


In [951]:
print(df.columns)
if 'GST_ActivityID' in df.columns:
    print(df['GST_ActivityID'].head(10))
else:
    print("Column 'GST_ActivityID' not found in Data Frame")

Index(['cmeID', 'startTime+cme', 'GST_ActivityID'], dtype='object')
3     2024-11-03T15:13:00-FLR-001
5     2024-11-04T01:05:00-FLR-001
6     2024-11-04T11:46:00-FLR-001
7     2024-11-04T15:26:00-FLR-001
8     2024-11-07T14:29:00-IPS-001
9     2024-11-05T06:35:00-FLR-001
12    2024-11-05T15:05:00-FLR-001
17    2024-11-06T15:50:00-FLR-001
19    2024-11-06T23:10:00-FLR-001
33    2024-11-10T11:51:00-FLR-001
Name: GST_ActivityID, dtype: object


In [952]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
df_filtered=df[df['GST_ActivityID'].str.contains('GST', na=False)]
print(f"Number of rows after filtering:{len(df_filtered)}")
print(df_filtered.head())


Number of rows after filtering:0
Empty DataFrame
Columns: [cmeID, startTime+cme, GST_ActivityID]
Index: []


### GST Data

In [953]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_requests = f'{base_url}GST?StartDate={startDate}&EndDate={endDate}&api_key={NASA_API_KEY}'
gst_requests

'https://api.nasa.gov/DONKI/GST?StartDate=2013-05-01&EndDate=2024-05-01&api_key=ludNWM2t3DZoKu7pHglwDRZslmnNHKa5FZHHTPyG'

In [954]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(cme_url)
gst_response

<Response [200]>

In [955]:
# Convert the response variable to json and store it as a variable named gst_json

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
gst_json = gst_response.json()
print(json.dumps(gst_json, indent=4))

[
    {
        "activityID": "2024-11-02T06:24:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2024-11-02T06:24Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "Faint, narrow CME visible to the SW in SOHO LASCO C2/C3 and STEREO A COR2. There are potential source candidate eruptions seen on the disk in available EUV imagery, including a faint dimming seen in GOES SUVI 195/284 and STEREO A EUV 195 around S40W10 starting at 2024-11-02T05:00Z. Dimming and moving field lines can also be seen south of AR 3869 (approx. S20W50) starting at 2024-11-02T02:52Z. The eruption is not seen in SDO AIA imagery due to a data gap from approximately 2024-11-01T22:00Z to 2024-11-02T0

In [956]:
# Convert gst_json to a Pandas DataFrame  

# Keep only the columns: activityID, startTime, linkedEvents
gst_df = pd.DataFrame(gst_json)
gst_df

gst_df= gst_df[['activityID', 'startTime', 'linkedEvents']]
gst_df.head()

,activityID,startTime,linkedEvents
0,2024-11-02T06:24:00-CME-001,2024-11-02T06:24Z,None
1,2024-11-02T09:12:00-CME-001,2024-11-02T09:12Z,None
2,2024-11-03T13:25:00-CME-001,2024-11-03T13:25Z,None
3,2024-11-03T15:27:00-CME-001,2024-11-03T15:27Z,[{'activityID': '2024-11-03T15:13:00-FLR-001'}]
4,2024-11-04T00:24:00-CME-001,2024-11-04T00:24Z,None


In [957]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset=["linkedEvents"])
gst_df.head()

,activityID,startTime,linkedEvents
3,2024-11-03T15:27:00-CME-001,2024-11-03T15:27Z,[{'activityID': '2024-11-03T15:13:00-FLR-001'}]
5,2024-11-04T01:36:00-CME-001,2024-11-04T01:36Z,[{'activityID': '2024-11-04T01:05:00-FLR-001'}...
6,2024-11-04T13:23:00-CME-001,2024-11-04T13:23Z,[{'activityID': '2024-11-04T11:46:00-FLR-001'}]
7,2024-11-04T17:24:00-CME-001,2024-11-04T17:24Z,[{'activityID': '2024-11-04T15:26:00-FLR-001'}...
8,2024-11-05T00:09:00-CME-001,2024-11-05T00:09Z,[{'activityID': '2024-11-07T14:29:00-IPS-001'}]


In [958]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
df_explode = gst_df.explode('linkedEvents', ignore_index=True).dropna()
df_explode.head()

,activityID,startTime,linkedEvents
0,2024-11-03T15:27:00-CME-001,2024-11-03T15:27Z,{'activityID': '2024-11-03T15:13:00-FLR-001'}
1,2024-11-04T01:36:00-CME-001,2024-11-04T01:36Z,{'activityID': '2024-11-04T01:05:00-FLR-001'}
2,2024-11-04T01:36:00-CME-001,2024-11-04T01:36Z,{'activityID': '2024-11-06T16:20:00-IPS-001'}
3,2024-11-04T13:23:00-CME-001,2024-11-04T13:23Z,{'activityID': '2024-11-04T11:46:00-FLR-001'}
4,2024-11-04T17:24:00-CME-001,2024-11-04T17:24Z,{'activityID': '2024-11-04T15:26:00-FLR-001'}


In [959]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:

df_explode.loc[:, "CME_ActivityID"] = df_explode['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
df_explode = df_explode.dropna(subset=['CME_ActivityID'])
print(df_explode.head())


                    activityID          startTime  \
0  2024-11-03T15:27:00-CME-001  2024-11-03T15:27Z   
1  2024-11-04T01:36:00-CME-001  2024-11-04T01:36Z   
2  2024-11-04T01:36:00-CME-001  2024-11-04T01:36Z   
3  2024-11-04T13:23:00-CME-001  2024-11-04T13:23Z   
4  2024-11-04T17:24:00-CME-001  2024-11-04T17:24Z   

                                    linkedEvents               CME_ActivityID  
0  {'activityID': '2024-11-03T15:13:00-FLR-001'}  2024-11-03T15:13:00-FLR-001  
1  {'activityID': '2024-11-04T01:05:00-FLR-001'}  2024-11-04T01:05:00-FLR-001  
2  {'activityID': '2024-11-06T16:20:00-IPS-001'}  2024-11-06T16:20:00-IPS-001  
3  {'activityID': '2024-11-04T11:46:00-FLR-001'}  2024-11-04T11:46:00-FLR-001  
4  {'activityID': '2024-11-04T15:26:00-FLR-001'}  2024-11-04T15:26:00-FLR-001  


In [960]:
df_explode.columns

Index(['activityID', 'startTime', 'linkedEvents', 'CME_ActivityID'], dtype='object')

In [961]:
# Convert the 'CME_ActivityID' column to string format 
#df_explode = df_explode['CME_ActivityID'].astype(str)
df_explode['CME_ActivityID'] = df_explode['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format 
#df_explode['activityID'].astype(str)
df_explode['activityID'] = df_explode['activityID'].astype(str)
# Convert startTime to datetime format  
df_explode['startTime'] = pd.to_datetime(df_explode['startTime'])
# Rename startTime to startTime_GST 
df_explode = df_explode.rename(columns={'startTime':'startTime+GST'})
# Drop linkedEvents
df_explode.drop(columns=['linkedEvents'], inplace=True)
# Verify that all steps were executed correctly
print(df_explode.head())





                    activityID             startTime+GST  \
0  2024-11-03T15:27:00-CME-001 2024-11-03 15:27:00+00:00   
1  2024-11-04T01:36:00-CME-001 2024-11-04 01:36:00+00:00   
2  2024-11-04T01:36:00-CME-001 2024-11-04 01:36:00+00:00   
3  2024-11-04T13:23:00-CME-001 2024-11-04 13:23:00+00:00   
4  2024-11-04T17:24:00-CME-001 2024-11-04 17:24:00+00:00   

                CME_ActivityID  
0  2024-11-03T15:13:00-FLR-001  
1  2024-11-04T01:05:00-FLR-001  
2  2024-11-06T16:20:00-IPS-001  
3  2024-11-04T11:46:00-FLR-001  
4  2024-11-04T15:26:00-FLR-001  


In [ ]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  

df_explode = df_explode[df_explode['activityID'].str.contains('CME', na=False)]
print(df_explode)

                     activityID             startTime+GST  \
0   2024-11-03T15:27:00-CME-001 2024-11-03 15:27:00+00:00   
1   2024-11-04T01:36:00-CME-001 2024-11-04 01:36:00+00:00   
2   2024-11-04T01:36:00-CME-001 2024-11-04 01:36:00+00:00   
3   2024-11-04T13:23:00-CME-001 2024-11-04 13:23:00+00:00   
4   2024-11-04T17:24:00-CME-001 2024-11-04 17:24:00+00:00   
5   2024-11-04T17:24:00-CME-001 2024-11-04 17:24:00+00:00   
6   2024-11-04T17:24:00-CME-001 2024-11-04 17:24:00+00:00   
7   2024-11-05T00:09:00-CME-001 2024-11-05 00:09:00+00:00   
8   2024-11-05T07:36:00-CME-001 2024-11-05 07:36:00+00:00   
9   2024-11-05T16:09:00-CME-001 2024-11-05 16:09:00+00:00   
10  2024-11-06T16:48:00-CME-001 2024-11-06 16:48:00+00:00   
11  2024-11-06T23:36:00-CME-001 2024-11-06 23:36:00+00:00   
12  2024-11-10T12:36:00-CME-001 2024-11-10 12:36:00+00:00   
13  2024-11-10T17:12:00-CME-001 2024-11-10 17:12:00+00:00   
14  2024-11-10T23:09:00-CME-001 2024-11-10 23:09:00+00:00   
15  2024-11-11T01:23:00-

### Merge both datatsets

In [963]:
# Now merge both datasets using 'gstID'/activityID and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(df_explode, df, left_on=['CME_ActivityID', 'activityID'], right_on=['GST_ActivityID','cmeID'], how='inner')
print(merged_df.head())

                    activityID             startTime+GST  \
0  2024-11-03T15:27:00-CME-001 2024-11-03 15:27:00+00:00   
1  2024-11-04T01:36:00-CME-001 2024-11-04 01:36:00+00:00   
2  2024-11-04T13:23:00-CME-001 2024-11-04 13:23:00+00:00   
3  2024-11-04T17:24:00-CME-001 2024-11-04 17:24:00+00:00   
4  2024-11-05T00:09:00-CME-001 2024-11-05 00:09:00+00:00   

                CME_ActivityID                        cmeID  \
0  2024-11-03T15:13:00-FLR-001  2024-11-03T15:27:00-CME-001   
1  2024-11-04T01:05:00-FLR-001  2024-11-04T01:36:00-CME-001   
2  2024-11-04T11:46:00-FLR-001  2024-11-04T13:23:00-CME-001   
3  2024-11-04T15:26:00-FLR-001  2024-11-04T17:24:00-CME-001   
4  2024-11-07T14:29:00-IPS-001  2024-11-05T00:09:00-CME-001   

              startTime+cme               GST_ActivityID  
0 2024-11-03 15:27:00+00:00  2024-11-03T15:13:00-FLR-001  
1 2024-11-04 01:36:00+00:00  2024-11-04T01:05:00-FLR-001  
2 2024-11-04 13:23:00+00:00  2024-11-04T11:46:00-FLR-001  
3 2024-11-04 17:24:00+00

In [969]:
# Verify that the new DataFrame has the same number of rows as cme and gst
display(merged_df.head())

,activityID,startTime+GST,CME_ActivityID,cmeID,startTime+cme,GST_ActivityID,timeDiff
0,2024-11-03T15:27:00-CME-001,2024-11-03 15:27:00+00:00,2024-11-03T15:13:00-FLR-001,2024-11-03T15:27:00-CME-001,2024-11-03 15:27:00+00:00,2024-11-03T15:13:00-FLR-001,0 days
1,2024-11-04T01:36:00-CME-001,2024-11-04 01:36:00+00:00,2024-11-04T01:05:00-FLR-001,2024-11-04T01:36:00-CME-001,2024-11-04 01:36:00+00:00,2024-11-04T01:05:00-FLR-001,0 days
2,2024-11-04T13:23:00-CME-001,2024-11-04 13:23:00+00:00,2024-11-04T11:46:00-FLR-001,2024-11-04T13:23:00-CME-001,2024-11-04 13:23:00+00:00,2024-11-04T11:46:00-FLR-001,0 days
3,2024-11-04T17:24:00-CME-001,2024-11-04 17:24:00+00:00,2024-11-04T15:26:00-FLR-001,2024-11-04T17:24:00-CME-001,2024-11-04 17:24:00+00:00,2024-11-04T15:26:00-FLR-001,0 days
4,2024-11-05T00:09:00-CME-001,2024-11-05 00:09:00+00:00,2024-11-07T14:29:00-IPS-001,2024-11-05T00:09:00-CME-001,2024-11-05 00:09:00+00:00,2024-11-07T14:29:00-IPS-001,0 days


### Computing the time it takes for a CME to cause a GST

In [965]:
print(merged_df.columns)

Index(['activityID', 'startTime+GST', 'CME_ActivityID', 'cmeID',
       'startTime+cme', 'GST_ActivityID'],
      dtype='object')


In [966]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = merged_df['startTime+GST'] - merged_df['startTime+cme']
print(merged_df[['startTime+GST', 'startTime+cme', 'timeDiff']].head())


              startTime+GST             startTime+cme timeDiff
0 2024-11-03 15:27:00+00:00 2024-11-03 15:27:00+00:00   0 days
1 2024-11-04 01:36:00+00:00 2024-11-04 01:36:00+00:00   0 days
2 2024-11-04 13:23:00+00:00 2024-11-04 13:23:00+00:00   0 days
3 2024-11-04 17:24:00+00:00 2024-11-04 17:24:00+00:00   0 days
4 2024-11-05 00:09:00+00:00 2024-11-05 00:09:00+00:00   0 days


In [967]:
# Use describe() to compute the mean and median time 
time_diff_stats = merged_df['timeDiff'].describe()
# that it takes for a CME to cause a GST. 
mean_time_diff = time_diff_stats['mean']
median_time_diff = merged_df['timeDiff'].median()
print(f"Mean time difference: {mean_time_diff}")
print(f"Median time difference: {mean_time_diff}")


Mean time difference: 0 days 00:00:00
Median time difference: 0 days 00:00:00


### Exporting data in csv format

In [968]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv, index=False')